# Compute i parameters

In [2]:
import brightway2 as bw
import pandas as pd
import os

# Set working directry
path = "."
os.chdir(path)

# Local functions
from utils.lookup_func import lookup_geothermal

# Set project
bw.projects.set_current("Geothermal")

# Retrieve activities
wellhead, diesel, steel, cement, water, \
drilling_mud, drill_wst, wells_closr, coll_pipe, \
plant, ORC_fluid, ORC_fluid_wst, diesel_stim, co2,_, _ = lookup_geothermal()
cooling_tower=bw.Database("geothermal energy").search("cooling tower")[0].key

list_act = [wellhead, diesel, steel, cement, water, 
         drilling_mud, drill_wst, wells_closr, coll_pipe,
         plant, cooling_tower, ORC_fluid, ORC_fluid_wst, diesel_stim]

# Retrieve methods 
ILCD_CC = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "climate change total" in str(method)]
ILCD_HH = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "human health" in str(method)]
ILCD_EQ = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "ecosystem quality" in str(method)]
ILCD_RE = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "resources" in str(method)]
ILCD = ILCD_CC + ILCD_HH + ILCD_EQ + ILCD_RE

# Calculate impact of activities
lca = bw.LCA({list_act[0]: 1}, ILCD[0])
lca.lci()
lca.lcia()
coeff = {}
for method in ILCD:
    s=[]
    lca.switch_method(method)
    for act in list_act:
        lca.redo_lcia({act: 1})
        s.append(lca.score)
    coeff[method] = s

# Retrieve CF for co2 emissions    
for method in ILCD:
    CFs = bw.Method(method).load()
    coeff[method].append(next((cf[1] for cf in CFs if cf[0]==co2),0))

# Build matrix
col_names = ["wellhead", "diesel", "steel", "cement", "water", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid", "ORC_fluid_wst", "diesel_stim", "co2"]
coeff_matrix = pd.DataFrame.from_dict(coeff, orient="index", columns=col_names)

# Re-arrange matrix
coeff_matrix["concrete"] = coeff_matrix["cement"] + coeff_matrix["water"] * 1/0.65
coeff_matrix["ORC_fluid_tot"] = coeff_matrix["ORC_fluid"] - coeff_matrix["ORC_fluid_wst"]
coeff_matrix["electricity_stim"] = coeff_matrix["diesel_stim"] * 3.6 / 0.3
coeff_matrix=coeff_matrix.drop(columns=["cement", "ORC_fluid", "ORC_fluid_wst", "diesel_stim"])

col_ord = ["wellhead", "diesel", "steel", "concrete", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid_tot", "water", "electricity_stim", "co2"]
coeff_matrix=coeff_matrix[col_ord]

is_= ["i1", "i2.1", "i2.2", "i2.3", "i2.4", "i2.5", \
      "i2.6","i3", "i4.1", "i4.2", "i4.3", "i5.1", "i5.2", "i6"]

if len(coeff_matrix.columns) == len(is_):
    coeff_matrix.columns=is_

In [4]:
os.path.abspath(path)

'E:\\Andrea\\OneDrive - University College London\\S4CE\\GSA\\Geothermal'

# Define symbolic

In [2]:
from sympy import symbols, collect, ratsimp, fraction

In [3]:
# Coefficients
i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6 = \
symbols('i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6')

# Number of wells With or without success rate coefficients
W_nT, W_nTe = symbols('W_nT, W_nTe')

# Other variables that are actually irrelevant, because they will be multiplied by 0 in the enhanced model
# I wanna keep them though for the sake of completeness
SW_n, S_w, S_el = symbols('SW_n, SW, S_el')

# Parameters for computing alpha and betas
E_co2, W_d, PW_ne = symbols('E_co2, W_d, PW_ne')


In [4]:
from cge_klausen import parameters
parameters.static()    

# Rename parameters (in accordance with parametric model)
P_ne     = parameters["installed_capacity"]
# PW_ne    = parameters["gross_power_per_well"]
PI_ratio = parameters["production_to_injection_ratio"]
SR_pi    = parameters["success_rate_primary_wells"]/100
D_i      = parameters["initial_harmonic_decline_rate"]
LT       = parameters["lifetime"]
SR_m     = parameters["success_rate_makeup_wells"]/100
SR_e     = parameters["success_rate_exploration_wells"]/100
D        = parameters["specific_diesel_consumption"]
C_S      = parameters["specific_steel_consumption"]
C_C      = parameters["specific_cement_consumption"]
DM       = parameters["specific_drilling_mud_consumption"]
# W_d      = parameters["average_depth_of_wells"]
CP       = parameters["collection_pipelines"]
CF       = parameters["capacity_factor"]
A_p      = parameters["auxiliary_power"]

# Constants
CT_el  = 864
CT_n   = 7/303.3
W_e_en = 3 * 0.3
OF = 0

class1 = ["climate change total"]

class2 = ["carcinogenic effects", "ionising radiation", "non-carcinogenic effects", 
          "ozone layer depletion", "photochemical ozone creation", "respiratory effects, inorganics",
          "freshwater and terrestrial acidification", "freshwater ecotoxicity", 
          "freshwater eutrophication", "marine eutrophication", "terrestrial eutrophication",
          "minerals and metals", "dissipated water", "fossils", "land use"]


In [5]:
# Supporting equations for CONVENTIONAL plant wells
W_Pn = P_ne/PW_ne             # production wells
W_In = P_ne/PW_ne / PI_ratio  # injection wells
W_Mn = W_Pn*D_i*LT            # makeup wells

# With success rate
a1 = (1 + 1/PI_ratio)/SR_pi + D_i*LT/SR_m
b1 = W_e_en/SR_e
W_nT__with_sr = W_Pn * a1  +  b1

# Total number without exploration
W_nTe_wout_expl = W_nT - W_e_en/SR_e

# Main equation

In [6]:
# Equation
nominator = W_nT*i1 \
          + W_d*W_nT * (D*i2_1 + C_S*i2_2 + C_C*i2_3  +  DM*i2_4 + i2_5 + i2_6) + \
          + W_nTe*CP*i3 \
          + P_ne*i4 \
          + SW_n*S_w * (i5_1 + S_el*i5_2)

c1 = P_ne*CF*(1-A_p)*LT*8760000
d1 = CT_el*CT_n*1000*30
denominator = c1  -  d1

summand = E_co2*i6

eq = nominator/denominator + summand
eq = eq.subs(i4, i4_1 + i4_2*CT_n + i4_3*OF)

In [7]:
eq

E_co2*i6 + 6.29186220504826e-11*SW*SW_n*(S_el*i5_2 + i5_1) + 6.29186220504826e-11*W_d*W_nT*(2200.0*i2_1 + 105.0*i2_2 + 40.0*i2_3 + 0.65*i2_4 + i2_5 + i2_6) + 6.29186220504826e-11*W_nT*i1 + 3.14593110252413e-8*W_nTe*i3 + 4.40430354353378e-9*i4_1 + 1.01648944295208e-10*i4_2

In [8]:
# Substitute variables in the main equation with supporting equations
eq_simplified_conventional_0 = \
eq.subs([(W_nTe, W_nTe_wout_expl), 
         (W_nT, W_nT__with_sr)])

In [9]:
eq_simplified_conventional_0

E_co2*i6 + 6.29186220504826e-11*SW*SW_n*(S_el*i5_2 + i5_1) + 6.29186220504826e-11*W_d*(1.33978019228699 + 277.636766510754/PW_ne)*(2200.0*i2_1 + 105.0*i2_2 + 40.0*i2_3 + 0.65*i2_4 + i2_5 + i2_6) + 6.29186220504826e-11*i1*(1.33978019228699 + 277.636766510754/PW_ne) + 4.40430354353378e-9*i4_1 + 1.01648944295208e-10*i4_2 + 8.73426138970409e-6*i3/PW_ne

## Compute alpha

In [10]:
PW_ne_val = parameters["gross_power_per_well"]   # set PW_ne to actual value
W_d_val   = parameters["average_depth_of_wells"] # set W_d to actual value

eq_simplified_conventional_1 = \
eq_simplified_conventional_0.subs([(PW_ne, PW_ne_val),
                                   (W_d, W_d_val)])

In [11]:
eq_simplified_conventional_1

E_co2*i6 + 6.29186220504826e-11*SW*SW_n*(S_el*i5_2 + i5_1) + 3.05119441393473e-9*i1 + 0.0151034123489769*i2_1 + 0.00072084468029208*i2_2 + 0.000274607497254126*i2_3 + 4.46237183037954e-6*i2_4 + 6.86518743135314e-6*i2_5 + 6.86518743135314e-6*i2_6 + 1.48344864519275e-6*i3 + 4.40430354353378e-9*i4_1 + 1.01648944295208e-10*i4_2

In [20]:
alpha_dict = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in class1:
        eq_simplified_conventional_alpha = eq_simplified_conventional_1.subs(is_dict)
        alpha1 = collect(eq_simplified_conventional_alpha, E_co2, evaluate=False)[E_co2]
        alpha2 = collect(eq_simplified_conventional_alpha, E_co2, evaluate=False)[1]
        alpha_dict[str(method)] = {'alpha1': alpha1,
                              'alpha2': alpha2}

In [21]:
alpha_dict

{"('ILCD 2.0 2018 midpoint no LT', 'climate change', 'climate change total')": {'alpha1': 1.00000000000000,
  'alpha2': 0.00456859175698260}}

In [22]:
# Example of the formula
eq_simplified_conventional_alpha

1.0*E_co2 + 0.0045685917569826

In [23]:
alpha_df = pd.DataFrame.from_dict(alpha_dict, orient='index')
alpha_df = alpha_df.astype(float)
alpha_df

,alpha1,alpha2
"('ILCD 2.0 2018 midpoint no LT', 'climate change', 'climate change total')",1.0,0.004569


# Compute betas

In [24]:
E_co2_val = parameters["co2_emissions"] # set E_co2 to actual value
eq_simplified_conventional_2 = \
eq_simplified_conventional_0.subs([(E_co2, E_co2_val)])

In [25]:
beta_dict = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in class2:
        eq_simplified_conventional_beta = eq_simplified_conventional_2.subs(is_dict)
        eq_simplified_conventional_beta = ratsimp(eq_simplified_conventional_beta)
        temp = collect(eq_simplified_conventional_beta, [W_d,PW_ne,W_d/PW_ne], evaluate=False)
        beta1 = collect(temp[list(temp.keys())[1]], W_d, evaluate=False)[W_d]
        beta2 = collect(temp[list(temp.keys())[1]], W_d, evaluate=False)[1]
        beta3 = temp[W_d]
        beta4 = temp[1]
        
        beta_dict[str(method)] = {'beta1': beta1,
                             'beta2': beta2,
                             'beta3': beta3,
                             'beta4': beta4}
        
        

In [26]:
temp

{W_d: 2.04559970225816e-13,
 1/PW_ne: 4.23900644433961e-11*W_d + 5.38252583081984e-8,
 1: 8.56116125386506e-9}

In [27]:
# Example of the formula
eq_simplified_conventional_beta

2.04559970225816e-13*W_d + 8.56116125386506e-9 + 1.0*(4.23900644433961e-11*W_d + 5.38252583081984e-8)/PW_ne

In [28]:
beta_df = pd.DataFrame.from_dict(beta_dict, orient='index')
beta_df = beta_df.astype(float)
beta_df

,beta1,beta2,beta3,beta4
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",1.061112e-12,1.181403e-09,5.120562e-15,7.268506e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation')",3.070379e-07,1.319461e-04,1.481660e-09,2.006216e-05
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",2.575297e-12,2.857134e-09,1.242750e-14,2.296825e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion')",1.024445e-12,3.489624e-10,4.943623e-15,6.986769e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation')",7.938542e-08,2.502208e-05,3.830869e-10,2.514107e-06
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",3.901732e-13,4.623747e-10,1.882842e-15,4.286987e-11
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification')",6.939912e-08,3.239950e-05,3.348964e-10,4.099331e-06
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",1.313228e-05,1.460457e-02,6.337189e-08,1.156160e-03
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",8.494234e-10,9.463493e-07,4.099027e-12,1.235788e-07
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'marine eutrophication')",2.553636e-08,6.042902e-06,1.232297e-10,7.702447e-07


# Save to excel

In [29]:
with pd.ExcelWriter(os.path.join(path, 'Simplified models coefficients conventional - symbolic.xlsx')) as writer:
    alpha_df.to_excel(writer, sheet_name='alpha')
    beta_df.to_excel(writer, sheet_name='beta')

# Compare with analytical solution

In [30]:
coeffs_=pd.read_excel(os.path.join(path, "Simplified models coefficients - analytical.xlsx"), sheet_name=["alpha", "beta", "chi", "delta"], index_col=0, dtype=object)
alpha_an = coeffs_["alpha"]
beta_an = coeffs_["beta"]

In [33]:
alpha_df-alpha_an

,alpha1,alpha2
"('ILCD 2.0 2018 midpoint no LT', 'climate change', 'climate change total')",0,-2.25011e-05


In [34]:
beta_df-beta_an

,beta1,beta2,beta3,beta4
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",-7.40443e-13,-4.1359e-25,-3.57313e-15,1.29247e-26
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation')",-5.44659e-09,2.71051e-20,-2.62834e-11,0
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",-2.29606e-12,-4.1359e-25,-1.108e-14,-2.58494e-26
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion')",-1.90047e-14,5.16988e-26,-9.171e-17,-1.29247e-26
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation')",-4.69612e-10,3.38813e-21,-2.26619e-12,4.23516e-22
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",-6.27475e-15,0,-3.02798e-17,6.46235e-27
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification')",-4.53419e-10,0,-2.18804e-12,0
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",-4.12723e-05,5.20417e-18,-1.99166e-07,0
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",-1.20136e-10,0,-5.79734e-13,2.64698e-23
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'marine eutrophication')",-1.43824e-10,-1.69407e-21,-6.94044e-13,-1.05879e-22
